### Getting started

In [34]:
import pandas as pd

In [35]:
df = pd.read_csv('./WA_Fn-UseC_-Telco-Customer-Churn.csv')

### Overview of data

In [36]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


View columns:

In [38]:
df.columns

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

Check distribution of churn vs stay:

In [39]:
df['Churn'].value_counts()

Churn
No     5174
Yes    1869
Name: count, dtype: int64

Verify that customers are unique.

In [40]:
df['customerID'].value_counts().sort_values(ascending=False)

customerID
7590-VHVEG    1
0280-XJGEX    1
7795-CFOCW    1
9237-HQITU    1
9305-CDSKC    1
             ..
2234-XADUH    1
4801-JZAZL    1
8361-LTMKD    1
1122-JWTJW    1
3186-AJIEK    1
Name: count, Length: 7043, dtype: int64

### Data cleaning - Converting "TotalCharges" to number and filling missing values

In [41]:
df['TotalCharges'].sort_values()

936           
3826          
4380          
753           
5218          
         ...  
6646    997.75
5598     998.1
3686    999.45
3353     999.8
2845     999.9
Name: TotalCharges, Length: 7043, dtype: object

In [42]:
df[['TotalCharges']].sort_values('TotalCharges').iat[0, 0]

' '

In [43]:
df['TotalCharges_Float'] = df.apply(lambda row: '0' if row['TotalCharges'] == ' ' else row['TotalCharges'], axis=1)
df['TotalCharges_Float'] = df.apply(lambda row: float(row['TotalCharges_Float']), axis=1)
df['TotalCharges_Float'] = df.apply(lambda row: row['tenure'] * row['MonthlyCharges'] if row['TotalCharges_Float'] == 0 else row['TotalCharges_Float'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   customerID          7043 non-null   object 
 1   gender              7043 non-null   object 
 2   SeniorCitizen       7043 non-null   int64  
 3   Partner             7043 non-null   object 
 4   Dependents          7043 non-null   object 
 5   tenure              7043 non-null   int64  
 6   PhoneService        7043 non-null   object 
 7   MultipleLines       7043 non-null   object 
 8   InternetService     7043 non-null   object 
 9   OnlineSecurity      7043 non-null   object 
 10  OnlineBackup        7043 non-null   object 
 11  DeviceProtection    7043 non-null   object 
 12  TechSupport         7043 non-null   object 
 13  StreamingTV         7043 non-null   object 
 14  StreamingMovies     7043 non-null   object 
 15  Contract            7043 non-null   object 
 16  Paperl

### Comparing churn by personal profile

In [44]:
df_churn = df[df['Churn'] == 'Yes'] 

In [45]:
df_stay = df[df['Churn'] == 'No']   

In [46]:
df.groupby(['Churn', 'gender'])['customerID'].count()

Churn  gender
No     Female    2549
       Male      2625
Yes    Female     939
       Male       930
Name: customerID, dtype: int64

Gender not an indication of churn. Also, the gender distribution of customers is uniform.

In [47]:
df.groupby(['Churn', 'SeniorCitizen'])['customerID'].count()

Churn  SeniorCitizen
No     0                4508
       1                 666
Yes    0                1393
       1                 476
Name: customerID, dtype: int64

Among those who stayed, around 15% were senior citizens, while among those who churned, around 35% were senior citizens. Therefore, being a senior citizen indicates greater likelihood of churning. This may be due to death.

In [48]:
df_partners = df.groupby(['Partner', 'Churn'])['customerID'].count().rename('count').reset_index()
df_partners['perc'] = df_partners.apply(lambda row: row['count'] / df_partners['count'].sum() * 100, axis=1)
df_partners.round(2)

,Partner,Churn,count,perc
0,No,No,2441,34.66
1,No,Yes,1200,17.04
2,Yes,No,2733,38.80
3,Yes,Yes,669,9.50


About half of customers have partners, and half don't. People without a partner are more likely to churn.

In [49]:
df_dep = df.groupby(['Dependents', 'Churn'])['customerID'].count().rename('count').reset_index()
df_dep['perc'] = df_dep.apply(lambda row: row['count'] / df_dep['count'].sum() * 100, axis=1)
df_dep.round(2)

,Dependents,Churn,count,perc
0,No,No,3390,48.13
1,No,Yes,1543,21.91
2,Yes,No,1784,25.33
3,Yes,Yes,326,4.63


More customers do not have dependents (nearly 70%). People without dependents are more likely to churn as well.

### Type of service (categorical variables)

In [50]:
service_list = ['PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod']
df_churn[service_list].mode()

,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod
0,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check


In [51]:
df_stay[service_list].mode()

,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod
0,Yes,No,DSL,No,Yes,No,No,Yes,Yes,Month-to-month,Yes,Mailed check


Variables possibly likely to favor churn (preliminary observations):
* Having multiple phone lines
* Having fiber optic internet
* Not having online backup
* Not having TV streaming
* Not having movie streaming
* Using electronic check as opposed to mailed check

### Verifying above hypotheses:

In [52]:
pd.melt(df_churn, id_vars=['Churn'], value_vars=service_list).value_counts().reset_index().sort_values(['variable', 'value'])

,Churn,variable,value,count
1,Yes,Contract,Month-to-month,1655
26,Yes,Contract,One year,166
34,Yes,Contract,Two year,48
7,Yes,DeviceProtection,No,1211
32,Yes,DeviceProtection,No internet service,113
15,Yes,DeviceProtection,Yes,545
18,Yes,InternetService,DSL,459
5,Yes,InternetService,Fiber optic,1297
30,Yes,InternetService,No,113
12,Yes,MultipleLines,No,849


In [53]:
pd.melt(df_stay, id_vars=['Churn'], value_vars=service_list).value_counts().reset_index().sort_values(['variable', 'value'])

,Churn,variable,value,count
4,No,Contract,Month-to-month,2220
28,No,Contract,One year,1307
20,No,Contract,Two year,1647
12,No,DeviceProtection,No,1884
26,No,DeviceProtection,No internet service,1413
13,No,DeviceProtection,Yes,1877
8,No,InternetService,DSL,1962
17,No,InternetService,Fiber optic,1799
24,No,InternetService,No,1413
2,No,MultipleLines,No,2541


Revisiting above conclusions:
* Having multiple phone lines - The values are very close, this is not significant
* Having fiber optic internet - This is significant
* Not having online backup - This is significant 
* Not having TV streaming - The values are very close, this is not significant
* Not having movie streaming - The values are very close, this is not significant
* Using electronic check as opposed to mailed check - This is significant

Additional contributors to churn:
* Having a month-to-month contract
* No device protection
* No online security or online backup
* Not having tech support
* Having paperless billing

In [54]:
df_churn.loc[df_churn['InternetService'] == 'Fiber optic'].loc[df_churn['PaymentMethod'] == 'Electronic check']['customerID'].count()

849

In [55]:
df_churn.loc[df_churn['InternetService'] == 'Fiber optic'].loc[df_churn['PaperlessBilling'] == 'Yes']['customerID'].count()

1068

Among customers who churned who have fiber optic internet (1297), 849 pay by electronic check (79% of those who pay by electronic check) and 1068 have paperless billing (76% of those who have paperless billing). These variables appear related, where electronic check/paperless billing seems to be a common payment method for fiber optic. This may be due to these being more convenient or safe ways to pay for this service. For our analysis, we can reduce these variables to the primary one--having fiber optic Internet.

To summarize the findings:
* In terms of Internet service, customers who stay have a fairly uniform distribution of DSL, fiber optic and no Internet services, whereas customers who churn are significantly more likely to have fiber optic than any other Internet service. 
  * The fiber optic services provided by the company are priced too high for the quality or there is severe competition from other service providers in terms of price and quality. Customers who have joined primarily for fiber optic services are more likely to churn than others.
  * Customers who prrioritize fiber optic Internet have different subscription and cancellation patterns for reasons unrelated to this company's services. Perhaps they require fiber optic internet seasonally.
* Customers who churn are more likely not to have online backup, online security, device protection, or tech support (these variables are more uniformly distributed among customers who stay). It appears that having one or more of these services increases likelihood of staying with the company.
* Customers who churn are significantly more likely to have month-to-month contracts than any other type (the distribution of contracts is more uniform among customers who stay).

To confirm this hypothesis, let us take six different samples of customers with a variety of services:
1. Customer has fiber optic internet, a month-to-month contract, and none of the related support services. They are most likely to churn.
2. Customer has fiber optic internet, a month-to-month contract, and some of the support services.
3. Customer has fiber optic internet, a yearly contract, and all of the support services.
4. Customer has DSL internet, a month-to-month contract, and none of the related support services. 
5. Customer has DSL internet, a month-to-month contract, and some of the support services.
6. Customer has DSL internet, a yearly contract, and all of the support services. They are least likely to churn. 

In [56]:
df[df['InternetService'] == 'Fiber optic'].loc[df['Contract'] == 'Month-to-month'].loc[df['OnlineBackup'] == 'No'].loc[df['OnlineSecurity'] == 'No'].loc[df['DeviceProtection'] == 'No'].loc[df['TechSupport'] == 'No']['Churn'].value_counts(normalize=True).mul(100).round(2).reset_index().sort_values('Churn')

,Churn,proportion
1,No,34.77
0,Yes,65.23


In [57]:
df[df['InternetService'] == 'Fiber optic'].loc[df['Contract'] == 'Month-to-month'].loc[df['OnlineBackup'] == 'No'].loc[df['OnlineSecurity'] == 'Yes'].loc[df['DeviceProtection'] == 'Yes'].loc[df['TechSupport'] == 'No']['Churn'].value_counts(normalize=True).mul(100).round(2).reset_index().sort_values('Churn')

,Churn,proportion
0,No,54.76
1,Yes,45.24


In [58]:
df[df['InternetService'] == 'Fiber optic'].loc[df['Contract'] == 'One year'].loc[df['OnlineBackup'] == 'Yes'].loc[df['OnlineSecurity'] == 'Yes'].loc[df['DeviceProtection'] == 'Yes'].loc[df['TechSupport'] == 'Yes']['Churn'].value_counts(normalize=True).mul(100).round(2).reset_index().sort_values('Churn')

,Churn,proportion
0,No,87.5
1,Yes,12.5


In [59]:
df[df['InternetService'] == 'DSL'].loc[df['Contract'] == 'Month-to-month'].loc[df['OnlineBackup'] == 'No'].loc[df['OnlineSecurity'] == 'No'].loc[df['DeviceProtection'] == 'No'].loc[df['TechSupport'] == 'No']['Churn'].value_counts(normalize=True).mul(100).round(2).reset_index().sort_values('Churn')

,Churn,proportion
0,No,55.53
1,Yes,44.47


In [60]:
df[df['InternetService'] == 'DSL'].loc[df['Contract'] == 'Month-to-month'].loc[df['OnlineBackup'] == 'No'].loc[df['OnlineSecurity'] == 'Yes'].loc[df['DeviceProtection'] == 'Yes'].loc[df['TechSupport'] == 'No']['Churn'].value_counts(normalize=True).mul(100).round(2).reset_index().sort_values('Churn')

,Churn,proportion
0,No,84.38
1,Yes,15.62


In [61]:
df[df['InternetService'] == 'DSL'].loc[df['Contract'] == 'One year'].loc[df['OnlineBackup'] == 'Yes'].loc[df['OnlineSecurity'] == 'Yes'].loc[df['DeviceProtection'] == 'Yes'].loc[df['TechSupport'] == 'Yes']['Churn'].value_counts(normalize=True).mul(100).round(2).reset_index().sort_values('Churn')

,Churn,proportion
0,No,91.84
1,Yes,8.16


The results confirm the hypothesized pattern:
* Customers with yearly contracts and at least some support services are less likely to churn. The probability of churn is further reduced if they are subscribed to DSL rather than fiber optic Internet.
* For customers with fiber optic Internet, churn can be significantly reduced by providing at least two support services (online backup, online security, device protection, tech support). Customers with all four of the above are very unlikely to churn irrespective of Internet service. Therefore, it is highly recommended to provide customers with at least two of these services, especially if they opt for fiber optic Internet.

### Tenure and payment (continuous variables)

In [62]:
payment_list = ['Churn', 'tenure', 'MonthlyCharges', 'TotalCharges_Float']
df[payment_list].groupby('Churn').agg(['mean', 'median', 'max', 'min'])

tenure                MonthlyCharges                         \
            mean median max min           mean  median     max    min   
Churn                                                                   
No     37.569965   38.0  72   0      61.265124  64.425  118.75  18.25   
Yes    17.979133   10.0  72   1      74.441332  79.650  118.35  18.85   

      TotalCharges_Float                            
                    mean    median      max    min  
Churn                                               
No           2549.911442  1679.525  8672.45   0.00  
Yes          1531.796094   703.550  8684.80  18.85

Tenure:
* The dataset goes back 6 years (72 months) as this is the maximum tenure present
* The average tenure of customers who have stayed is higher. However, even among customers who churned, at least one had a tenure  of 72 months, indicating the some churn after a long stay with the company.

Average Monthly Charges:
* Customers who churn have higher average monthly charges. This may be related to the observation about fiber optic services, since fiber optic is usually more expensive than DSL. This is bad news for the company, since it seems worse at retaining higher paying customers. 

Total Charges:
* This appears to be a product (not exact) of the above two. Therefore I won't consider it separately. 

In [63]:
df.groupby('InternetService')['MonthlyCharges'].mean()

InternetService
DSL            58.102169
Fiber optic    91.500129
No             21.079194
Name: MonthlyCharges, dtype: float64

It is confirmed that fiber optic Internet is on average more expensive than DSL, therefore, the observations about mean monthly charges are probably related to having this service. Therefore, the pattern observed here does not seem to be an independent contributing factor to churn.

### Which support service is the most important?

In [64]:
df_2 = df_churn.loc[df_churn['InternetService'] == 'Fiber optic'].loc[df_churn['Contract'] == 'Month-to-month']
df_2[['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport']].value_counts().rename('number_who_churned').reset_index()

,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,number_who_churned
0,No,No,No,No,531
1,No,No,Yes,No,148
2,No,Yes,No,No,147
3,No,Yes,Yes,No,99
4,Yes,No,No,No,57
5,No,No,No,Yes,39
6,No,No,Yes,Yes,32
7,Yes,No,Yes,No,19
8,Yes,Yes,No,No,19
9,No,Yes,No,Yes,18


While it is clear that these services work in tandem to reduce churn rate, the most impactful service to add was tech support, followed by online security, then online backup, then device protection. Interestingly, the lowest churn categories only had three of the four services. It is not necessary to add all four services. It seems that the best way to reduce churn is to add tech support and online security. In the category of people who had these two services only, only 9 churned. The value of adding more services is small.